In [1]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
from stable_baselines3 import SAC, HerReplayBuffer
from environments import *

import torch
import numpy as np
import gym
import imageio

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\torch\utils\tensorboard\__init__.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion


In [2]:
env_id = 'G1ControlEnv-v1'
video_folder = 'renders'
video_length = 500

In [3]:
models = {
    "g1-control-direction": "trained_models/g1-control/direction",
    "g1-control-orientation": "trained_models/g1-control/orientation",
    "g1-control-direction+orientation": "trained_models/g1-control/direction+orientation",
}

In [4]:
env = gym.make(env_id)

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [7]:
for name, model_path in models.items():

    model = SAC.load(model_path)

    for j, task in enumerate(env.sample_tasks(10)):
        env.reset_task(task)
        env.reset()

        obs = env.reset()
        img = env.render(mode='rgb_array')

        images = []

        for i in range(video_length + 1):
            images.append(img)
            action = model.predict(obs)
            obs, _, _, _ = env.step(action[0])
            img = env.render(mode='rgb_array')

        imageio.mimsave(
        'renders/' + name + '_' + str(j) + '.gif',
            [np.array(img) for i, img in enumerate(images) if i%2 == 0], 
            fps=29,
        )


    env.close()

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\core.py:51: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


KeyboardInterrupt: 